# Devoir 5, question 1

Considérons le problème
\begin{align*}
\min\ & 10x_1 + 3x_2 + 7x_3 + 4x_4 \\
\mbox{t.q. }
& 2x_1 + x_2 + 2x_3 + x_4 = 7 \\
& x_1 + x_2 = 3 \\
& x_3 + 2x_4 \leq 5 \\  
& x_3 + x_4 = 3 \\  
& x_1 \geq 0, x_2 \geq 0, x_3 \geq 0, x_4 \geq 0.
\end{align*}

Nous pouvons le réécrire sous forme standard en introduisant la variable d'écart $x_5$:
\begin{align*}
\min\ & 10x_1 + 3x_2 + 7x_3 + 4x_4 \\
\mbox{t.q. }
& 2x_1 + x_2 + 2x_3 + x_4 = 7 \\
& x_1 + x_2 = 3 \\
& x_3 + 2x_4 + x_5 = 5 \\  
& x_3 + x_4 = 3 \\  
& x_1 \geq 0, x_2 \geq 0, x_3 \geq 0, x_4 \geq 0, x_5 \geq 0.
\end{align*}

Nous allons tout d'abord le résoudre en employant Gurobi.

In [1]:
using JuMP, LinearAlgebra

In [2]:
using Gurobi

In [3]:
c = [ 10; 3; 7 ; 4; 0 ]
A = [2 1 2 1 0 ;
     1 1 0 0 0 ;
     0 0 1 2 1 ;
     0 0 1 1 0 ]
b = [ 7; 3; 5; 3 ]

n = length(c)

5

In [4]:
m = Model(with_optimizer(Gurobi.Optimizer))

@variable(m, x[1:5] >= 0)

for k = 1:4
    @constraint(m, sum(A[k,i]*x[i] for i = 1:n) == b[k])
end

@objective(m, Min, sum(x[i]*c[i] for i = 1:n))

print(m)

Academic license - for non-commercial use only
Min 10 x[1] + 3 x[2] + 7 x[3] + 4 x[4]
Subject to
 2 x[1] + x[2] + 2 x[3] + x[4] == 7.0
 x[1] + x[2] == 3.0
 x[3] + 2 x[4] + x[5] == 5.0
 x[3] + x[4] == 3.0
 x[1] >= 0.0
 x[2] >= 0.0
 x[3] >= 0.0
 x[4] >= 0.0
 x[5] >= 0.0


In [5]:
status = optimize!(m)

Academic license - for non-commercial use only
Optimize a model with 4 rows, 5 columns and 11 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 7e+00]
Presolve removed 4 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  2.400000000e+01


In [6]:
value.(x)

5-element Array{Float64,1}:
 0.0
 3.0
 1.0
 2.0
 0.0

Calculons la solution duale associée.

In [7]:
list_of_constraint_types(m)

2-element Array{Tuple{DataType,DataType},1}:
 (GenericAffExpr{Float64,VariableRef}, MathOptInterface.EqualTo{Float64})
 (VariableRef, MathOptInterface.GreaterThan{Float64})

In [8]:
λ = dual.(all_constraints(m, AffExpr, MOI.EqualTo{Float64}))

4-element Array{Float64,1}:
 3.0
 0.0
 0.0
 1.0

Calculons les variables d'écart associées aux contraintes duales:

In [9]:
s = c - transpose(A)*λ

5-element Array{Float64,1}:
 4.0
 0.0
 0.0
 0.0
 0.0

Nous pouvons vérifier que les contraintes de complémentarité sont satisfaites en sommant les variables primales avec les variables d'écart duales

In [10]:
sum(value(x[i])*s[i] for i in 1:n)

0.0

Il en va de même pour la dualité forte:

In [11]:
dot(c, value.(x))-dot(b, λ)

0.0

Effectuons à présent le primal-dual à la main. Comme $c$ est non-négatif, $(0,0,0,0)$ est dual-réalisable, avec comme variables d'écart initiales

In [12]:
s = c - transpose(A)*zeros(4)

5-element Array{Float64,1}:
 10.0
  3.0
  7.0
  4.0
  0.0

Le tableau du simplexe associé au problème artificiel est

In [13]:
Art = [[ A I b ] ; [ zeros(1,n) ones(1,4) 0 ]]

5×10 Array{Float64,2}:
 2.0  1.0  2.0  1.0  0.0  1.0  0.0  0.0  0.0  7.0
 1.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  3.0
 0.0  0.0  1.0  2.0  1.0  0.0  0.0  1.0  0.0  5.0
 0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0  3.0
 0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0

Nous devons annuler les coûts réduits des variables artificielles, qui agissent pour l'instant comme variables de base.

In [14]:
Art[5,:] = Art[5,:] - sum(Art[i,:] for i = 1:4)
Art

5×10 Array{Float64,2}:
  2.0   1.0   2.0   1.0   0.0  1.0  0.0  0.0  0.0    7.0
  1.0   1.0   0.0   0.0   0.0  0.0  1.0  0.0  0.0    3.0
  0.0   0.0   1.0   2.0   1.0  0.0  0.0  1.0  0.0    5.0
  0.0   0.0   1.0   1.0   0.0  0.0  0.0  0.0  1.0    3.0
 -3.0  -2.0  -4.0  -4.0  -1.0  0.0  0.0  0.0  0.0  -18.0

Nous reprenons la fonction de pivotage pour mettre à jour le tableau du simplexe.

In [15]:
function pivot!(A, i::Int, j::Int)
    (m, n) = size(A)
    A[i, :] = A[i, :] / A[i, j]
    for k in 1:m
        if k != i
            A[k, :] -= A[i, :]*A[k, j]
        end
    end
    return A
end

pivot! (generic function with 1 method)

Nous initialisons l'ensemble $P$ en cherchant les éléments nuls.

In [16]:
P = findall(x -> x == 0.0, s)

1-element Array{Int64,1}:
 5

Ainsi $P = \{ 5 \}$ et comme le coût réduit associé à $x_5$ est négatif, nous faisons entrer $x_5$ dans la base. En observant le tableau, nous observons que la pivot se trouve en (3,5).

In [17]:
pivot!(Art, 3, 5)

5×10 Array{Float64,2}:
  2.0   1.0   2.0   1.0  0.0  1.0  0.0  0.0  0.0    7.0
  1.0   1.0   0.0   0.0  0.0  0.0  1.0  0.0  0.0    3.0
  0.0   0.0   1.0   2.0  1.0  0.0  0.0  1.0  0.0    5.0
  0.0   0.0   1.0   1.0  0.0  0.0  0.0  0.0  1.0    3.0
 -3.0  -2.0  -3.0  -2.0  0.0  0.0  0.0  1.0  0.0  -13.0

Nous mettons à jour les variables d'écart duales. Commençons par chercher l'élément pivot.

In [18]:
idx = findall(x -> x < 0, Art[5,1:5])
idx[argmin(s[idx]./-Art[5,idx])]

2

Nous pivotons donc sur la deuxième colonne, et les nouvelles variables d'écart duales sont

In [19]:
s = s-s[2]/Art[5,2]*Art[5,1:5]

5-element Array{Float64,1}:
 5.5
 0.0
 2.5
 1.0
 0.0

$P$ est à présent

In [20]:
P = [2,5]

2-element Array{Int64,1}:
 2
 5

Nous pouvons donc faire entrer $x_2$ dans la base, et l'examen du tableau nous montre qu'en pivote sur l'élément (2,2).

In [21]:
pivot!(Art, 2, 2)

5×10 Array{Float64,2}:
  1.0  0.0   2.0   1.0  0.0  1.0  -1.0  0.0  0.0   4.0
  1.0  1.0   0.0   0.0  0.0  0.0   1.0  0.0  0.0   3.0
  0.0  0.0   1.0   2.0  1.0  0.0   0.0  1.0  0.0   5.0
  0.0  0.0   1.0   1.0  0.0  0.0   0.0  0.0  1.0   3.0
 -1.0  0.0  -3.0  -2.0  0.0  0.0   2.0  1.0  0.0  -7.0

Nous continuons la procédure jusqu'à convergence, ou détection de problème non borné.

In [22]:
idx = findall(x -> x < 0, Art[5,1:5])
i = idx[argmin(s[idx]./-Art[5,idx])]
s = s-s[i]/Art[5,i]*Art[5,1:5]

5-element Array{Float64,1}:
 5.0
 0.0
 1.0
 0.0
 0.0

In [23]:
pivot!(Art, 3, 4)

5×10 Array{Float64,2}:
  1.0  0.0   1.5  0.0  -0.5  1.0  -1.0  -0.5  0.0   1.5
  1.0  1.0   0.0  0.0   0.0  0.0   1.0   0.0  0.0   3.0
  0.0  0.0   0.5  1.0   0.5  0.0   0.0   0.5  0.0   2.5
  0.0  0.0   0.5  0.0  -0.5  0.0   0.0  -0.5  1.0   0.5
 -1.0  0.0  -2.0  0.0   1.0  0.0   2.0   2.0  0.0  -2.0

In [24]:
idx = findall(x -> x < 0, Art[5,1:5])
i = idx[argmin(s[idx]./-Art[5,idx])]
s = s-s[i]/Art[5,i]*Art[5,1:5]
i

3

In [25]:
pivot!(Art, 1, 3)

5×10 Array{Float64,2}:
  0.666667  0.0  1.0  0.0  -0.333333  …  -0.666667  -0.333333  0.0  1.0
  1.0       1.0  0.0  0.0   0.0           1.0        0.0       0.0  3.0
 -0.333333  0.0  0.0  1.0   0.666667      0.333333   0.666667  0.0  2.0
 -0.333333  0.0  0.0  0.0  -0.333333      0.333333  -0.333333  1.0  0.0
  0.333333  0.0  0.0  0.0   0.333333      0.666667   1.33333   0.0  0.0

La fonction artificielle prend la valeur 0 et nous avons donc convergé. La solution optimale est
$x_1 = 0$, $x_2 = 3$, $x_3 = 1$, $x_4 = 2$, $x_5 = 0$, ce qui correspond bien à ce que nous avait donné JuMP.

Remarquons cependant que nous avons encore une variable artificielle, $y_4$, dans la base. Nous pouvons l'échanger avec $x_1$ ou $x_5$.

In [26]:
C = Art
pivot!(Art, 4, 1)

5×10 Array{Float64,2}:
 0.0   0.0   1.0   0.0  -1.0           0.0   0.0  -1.0   2.0   1.0
 0.0   1.0   0.0   0.0  -1.0          -1.0   2.0  -1.0   3.0   3.0
 0.0   0.0   0.0   1.0   1.0           0.0   0.0   1.0  -1.0   2.0
 1.0  -0.0  -0.0  -0.0   1.0           1.0  -1.0   1.0  -3.0  -0.0
 0.0   0.0   0.0   0.0   5.55112e-17   1.0   1.0   1.0   1.0   0.0

In [27]:
Art = C
pivot!(Art, 4, 5)

5×10 Array{Float64,2}:
  1.0           0.0   1.0   0.0  0.0   1.0  -1.0  0.0  -1.0   1.0
  1.0           1.0   0.0   0.0  0.0   0.0   1.0  0.0   0.0   3.0
 -1.0           0.0   0.0   1.0  0.0  -1.0   1.0  0.0   2.0   2.0
  1.0          -0.0  -0.0  -0.0  1.0   1.0  -1.0  1.0  -3.0  -0.0
 -5.55112e-17   0.0   0.0   0.0  0.0   1.0   1.0  1.0   1.0   0.0

L'expression de la solution en termes de base n'est pas unique, en raison de sa dégénérescence. Remarquons également que le vecteur de variables d'écart duales est lui différent, ce qui suggère que la solution duale n'est pas unique

In [28]:
s

5-element Array{Float64,1}:
 4.5
 0.0
 0.0
 0.0
 0.5

Essayons de trouver à la main $\lambda$ à partir des écarts de complémentarité, comme
$$
A'\lambda + s = c
$$
Nous avons

In [29]:
c-s

5-element Array{Float64,1}:
  5.5
  3.0
  7.0
  4.0
 -0.5

In [30]:
transpose(A)

5×4 Transpose{Int64,Array{Int64,2}}:
 2  1  0  0
 1  1  0  0
 2  0  1  1
 1  0  2  1
 0  0  1  0

On en tire $\lambda_1 = 2.5$, $\lambda_2 = 0.5$, $\lambda_3 = -0.5$, $\lambda_4 = 2.5$, ce qui est différent de la solution duale donnée par Gurobi. Pour autant, cette solution est réalisable pour le dual et satisfait la dualité forte:

In [31]:
λ2 = [ 2.5; 0.5; -0.5; 2.5]
transpose(A)*λ2 + s - c

5-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [32]:
dot(λ2,b)-dot(c,value.(x))

0.0